https://github.com/PlayVoice/so-vits-svc-5.0/

↑原仓库

*《colab保持连接的方法》*https://zhuanlan.zhihu.com/p/144629818

预览版本，可使用预设模型进行推理

# **环境配置&必要文件下载**


In [ ]:
#@title 看看抽了个啥卡~~基本都是T4~~
!nvidia-smi

In [ ]:
#@title 克隆github仓库
!git clone https://github.com/PlayVoice/so-vits-svc-5.0/ -b bigvgan-mix-v2

In [ ]:
#@title 安装依赖&下载必要文件
%cd /content/so-vits-svc-5.0

!pip install -r requirements.txt
!pip install --upgrade pip setuptools numpy numba

!wget -P whisper_pretrain/ https://openaipublic.azureedge.net/main/whisper/models/81f7c96c852ee8fc832187b0132e569d6c3065a3252ed18e56effd0b6a73e524/large-v2.pt
!wget -P speaker_pretrain/ https://github.com/PlayVoice/so-vits-svc-5.0/releases/download/dependency/best_model.pth.tar
!wget -P crepe/assets https://github.com/PlayVoice/so-vits-svc-5.0/releases/download/dependency/full.pth
!wget -P vits_pretrain https://github.com/PlayVoice/so-vits-svc-5.0/releases/download/5.0/sovits5.0.pretrain.pth

In [ ]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

# 包含多说话人的推理预览

In [ ]:
#@title 提取内容编码

#@markdown **将处理好的" .wav "输入源文件上传到云盘根目录，并修改以下选项**

#@markdown **" .wav "文件【文件名】**
input = "\u30AE\u30BF\u30FC\u3068\u5B64\u72EC\u3068\u84BC\u3044\u60D1\u661F" #@param {type:"string"}
input_path = "/content/drive/MyDrive/"
input_name =  input_path + input
!PYTHONPATH=. python whisper/inference.py -w {input_name}.wav -p test.ppg.npy

In [ ]:
#@title 推理

#@markdown **将处理好的" .wav "输入源文件上传到云盘根目录，并修改以下选项**

#@markdown **" .wav "文件【文件名】**
input = "\u30AE\u30BF\u30FC\u3068\u5B64\u72EC\u3068\u84BC\u3044\u60D1\u661F" #@param {type:"string"}
input_path = "/content/drive/MyDrive/"
input_name =  input_path + input
#@markdown **指定说话人（0001~0056）（推荐0022、0030、0047、0051）**
speaker = "0002" #@param {type:"string"}
!PYTHONPATH=. python svc_inference.py --config configs/base.yaml --model vits_pretrain/sovits5.0.pretrain.pth --spk ./configs/singers/singer{speaker}.npy --wave {input_name}.wav  --ppg test.ppg.npy

推理结果保存在根目录，文件名为svc_out.wav

# 训练

将音频剪裁为小于30秒的音频段，响度匹配并修改为单声道，预处理时会进行重采样所以对采样率无要求。（但是降低采样率的操作会降低你的数据质量）

**使用Adobe Audition™的响度匹配功能可以一次性完成重采样修改声道和响度匹配。**

之后将音频文件保存为以下文件结构：
```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav
```

打包为zip格式，命名为data.zip，上传到网盘根目录。

In [ ]:
#@title 从云盘获取数据集
!unzip -d /content/so-vits-svc-5.0/ /content/drive/MyDrive/data.zip #自行修改路径与文件名

In [ ]:
#@title 重采样
# 生成采样率16000Hz音频, 存储路径为：./data_svc/waves-16k
!python prepare/preprocess_a.py -w ./dataset_raw -o ./data_svc/waves-16k -s 16000
# 生成采样率32000Hz音频, 存储路径为：./data_svc/waves-32k
!python prepare/preprocess_a.py -w ./dataset_raw -o ./data_svc/waves-32k -s 32000

In [ ]:
#@title 提取f0
!python prepare/preprocess_f0.py -w data_svc/waves-16k/ -p data_svc/pitch

In [ ]:
#@title 使用16k音频，提取内容编码
!PYTHONPATH=. python prepare/preprocess_ppg.py -w data_svc/waves-16k/ -p data_svc/whisper

In [ ]:
#@title 使用16k音频，提取内容编码
!PYTHONPATH=. python prepare/preprocess_hubert.py -w data_svc/waves-16k/ -v data_svc/hubert

In [ ]:
#@title 提取音色特征
!PYTHONPATH=. python prepare/preprocess_speaker.py data_svc/waves-16k/ data_svc/speaker

In [ ]:
#（解决“.ipynb_checkpoints”相关的错）
!rm -rf "find -type d -name .ipynb_checkpoints"

In [ ]:
#（解决“.ipynb_checkpoints”相关的错）
!rm -rf .ipynb_checkpoints
!find . -name ".ipynb_checkpoints" -exec rm -rf {} \;

In [ ]:
#@title 提取平均音色
!PYTHONPATH=. python prepare/preprocess_speaker_ave.py data_svc/speaker/ data_svc/singer

In [ ]:
#@title 提取spec
!PYTHONPATH=. python prepare/preprocess_spec.py -w data_svc/waves-32k/ -s data_svc/specs

In [ ]:
#@title 生成索引
!python prepare/preprocess_train.py

In [ ]:
#@title 训练文件调试
!PYTHONPATH=. python prepare/preprocess_zzz.py

In [ ]:
#@title  设定模型备份
#@markdown **是否备份模型到云盘，colab随时爆炸建议备份，默认保存到云盘根目录Sovits5.0文件夹**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !mkdir -p /content/so-vits-svc-5.0/chkpt/
  !rm -rf /content/so-vits-svc-5.0/chkpt/
  !mkdir -p /content/drive/MyDrive/Sovits5.0
  !ln -s /content/drive/MyDrive/Sovits5.0 /content/so-vits-svc-5.0/chkpt/

In [ ]:
#@title  开始训练
%load_ext tensorboard
%tensorboard --logdir /content/so-vits-svc-5.0/logs/

!PYTHONPATH=. python svc_trainer.py -c configs/base.yaml -n sovits5.0